In [ ]:
# initial setup for any notebook

%load_ext autoreload
%autoreload 2
import sys
sys.path.append("/home/yarcoh/projects/thesis-code4") # go to parent dir

In [ ]:
import numpy as np
from ModulationPy import ModulationPy
from commpy import rrcosfilter
from matplotlib import pyplot as plt

from src.optics.split_step_fourier import SplitStepFourier
from src.general_methods.visualizer import Visualizer
from src.optics.myFNFTpy.FNFTpy import nsev_inverse_xi_wrapper, nsev_inverse, nsev

### 0) input message:
we choose number of bits based on the number of symbols we want

so, number of bits: $ N_b = N_s*N_{sps} = 2048 $
where $ N_{sps} = log_2(M_{QAM}) = 4 $ is the num of samples per symbol

we get:
$ b=[b_0, b_1, ..., b_{N_b}] : b_i\in{0,1} \; \forall i$

In [ ]:
m_qam = 16
num_symbols = 512
sps = int(np.log2(m_qam))
length_of_msg = num_symbols*sps
x0 = np.random.choice([0, 1], size=length_of_msg)

# plot
Visualizer.print_bits(x0, sps, 'message before channel')

### 1) Modulation
$ s = f_{MOD}(b) = [s_0, s_1, ... , s_{N_s}] : s_i = a_i + jb_i $

s.t:
$ f_{MOD}:B^{N_s*N_{sps}} \rightarrow C^{N_s}  $
$ N_s = 512 $

In [ ]:
modem = ModulationPy.QAMModem(m_qam, soft_decision=False)
x1 = modem.modulate(x0)

# plot
Visualizer.plot_constellation_map_with_points(x1, m_qam, 'clean before channel')
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(11, 4))
Visualizer.my_plot(np.real(x1[0:30]), function='stem', name='i (real)', ax=ax1, hold=1)
Visualizer.my_plot(np.imag(x1[0:30]), function='stem', name='q (imag)', ax=ax2)
print(f'num of symbols = {len(x1)}')

### 2) over sampling

$ \hat s = [s_0, [0, 0, ..., 0], s_1, [0, 0, ..., 0], ..., s_{\hat N}, [0, 0, ..., 0]] $

s.t:

$ \hat s \in R^{\hat N} : \hat N = N_s*N_{os}= 4096 $ <br>
$ N_{os} = 8 $

In [ ]:
over_sampling = 8  # sampling rate

x2 = np.zeros(over_sampling*len(x1), dtype=np.complex64)
x2[::over_sampling] = x1

# plot
Visualizer.my_plot(np.real(x2[0:50]), name='zero padded - i (real)', function='stem')
print(f'vec length = {len(x2)}, over_sampling period = {over_sampling}')

### 3) Pulse shaping
$ X[\xi] = h \circledast s  $

length of h, $N_h$ is chosen such that the length of X, $N_X$ will be the next closest power of 2
* (unless its too small [less than 100], so we skip to the next closest power of 2)

for $ \hat N = 4096 $ we chose $ N_h = 4097 $
and we get $ N_X = 8192 $

In [ ]:
roll_off = 0.25  # filter roll-off factor
Ts = 3000  # symbol period

lenx = len(x2)
desired_len = int(2 ** np.ceil(np.log2(lenx)))  # the next power of 2
if desired_len - lenx < 100:
    desired_len = int(2 ** (1 + np.ceil(np.log2(lenx))))  # the next next power of 2

N_rrc = int(np.ceil(desired_len - lenx)) + 1
alpha: float = roll_off  # default = 0.25
fs = over_sampling/Ts
h_ind, h_rrc = rrcosfilter(N_rrc, alpha, Ts, fs)
x3 = np.convolve(h_rrc, x2)  # Waveform with PSF

# xi axis should be 1/ over sampling

#plot
zm = range(N_rrc//2, N_rrc//2 + 24)
# zm = range(2000, 2000 + 24)
Visualizer.twin_zoom_plot('|h_rrc filter|^2', np.abs(h_rrc) ** 2, zm)
# Visualizer.twin_zoom_plot('real{X(xi) * h(xi)}', np.real(x3), range(0, N_rrc))
Visualizer.twin_zoom_plot('real{X(xi) * h(xi)}', np.real(x3), range(2000, 2024))

print(f'filter len = {N_rrc}, signal len = {len(x3)}')

### 4) pre-equalize
 - normalize by $\mu$:   $ \hat X(\xi) = X \cdot A_\mu$
 - scaling amplitude to [-1,1]:
   - $ A = |\hat X(\xi)| $
   - $ \phi = \angle \hat X(\xi) $
   - $ \hat{A} = \sigma(A) $
   - $ \tilde {X} = \hat{A} e^{j\phi} $


In [ ]:
normalization_factor: float = 0.32
# normalization_factor: float = 1e3
x41 = normalization_factor*x3
#scaling
    #get amp_phase
amp = np.abs(x41)
phase = np.angle(x41)
    # sigmoid
    # TODO: take the scaling function from stas' doc
scaled_amp = 2 / (1 + np.exp(-amp)) - 1
    # get complex from amp_phase
x4 = scaled_amp * np.exp(1j * phase)

# plot
Visualizer.double_plot('real',np.real(x41),np.real(x4),None,None,'normalized signal','scaled signal')
Visualizer.double_plot('real',np.real(x41),np.real(x4),xivec/8,xivec/8,'normalized signal','scaled signal')

### 5) INFT

$ x(t) = \mathcal{F}^{-1}\{\hat X\} $

with the following params:
* $ N_\xi = N_{sps} \cdot N_{os} = len(\hat X) $
* $ N_t = 2 ^ {\lfloor log_2(N_\xi) \rfloor}$
* $ t = [\frac{-N_t}{2}:\frac{N_t}{2}] * dt $

In [ ]:
# generate INFT params:
N_xi = len(x3)
N_time = int(2 ** np.floor(np.log2(N_xi)))  # (=D)
dt = 1/N_time#e-12
# dt = dt / t0 # TODO:
tvec = np.arange(start=-N_time/2, stop=N_time/2)*dt  # np.linspace(-t0, t0, N_time)
rv, xi = nsev_inverse_xi_wrapper(N_time, tvec[0], tvec[-1], N_xi, display_c_msg=True)
xivec = xi[0] + np.arange(N_xi)*(xi[1] - xi[0])/(N_xi - 1)
dxi= xivec[1] - xivec[0]
BW = xivec.max()/8

#plot
print(
    f'dxi = {dxi/1:.2f} Hz\n'
    f'xi ∈ [{xivec[0]/1:.2f}:{xivec[-1]/1:.2f}] Hz ,\t N_xi   (=M) = {N_xi}\n'
    f't  ∈ [{tvec[0]*1:.2f}:{tvec[-1]*1:.2f}] s  ,\t N_time (=D) = {N_time}\n'
    f'BW = {BW/1:.2f} Hz'
)
# Visualizer.twin_zoom_plot('real{X(xi)}', np.real(x4), range(4000, 4200), xivec, xlabel='xi')


In [ ]:
xi

In [ ]:
# INFT
contspec = x4
bound_states = []  # np.array([0.7j, 1.7j])
discspec = []  # [1.0, -1.0]

cst = 1  # continuous spectrum type - default is None
dst = 0  # default is None

res = nsev_inverse(xivec, tvec, contspec, bound_states, discspec,
                   cst=cst, dst=dst, display_c_msg=True)

assert res['return_value'] == 0, "INFT failed"
x5 = res['q']  # q[t,0]

# plot
print(f'length of INFT(x) = {len(x5)}')
Visualizer.print_signal_specs(x5, tvec)
Visualizer.twin_zoom_plot('real{X(t)}', np.real(x5), range(4000, 4200), tvec, 't')

### 6) SSF

now we go through the SSF channel
which has N iterations

#### algorithm:
* $a\in \mathbb{C}^{N_t}$
* $a = a \cdot \sqrt{P_0}$
* $w = [0,...,\frac{N_t}{2}-1,\frac{-N_t}{2},...,-1] \cdot \frac{2 \pi}{N_t \cdot dt}$
* $ v_1 = \exp\left(\frac{j \cdot \beta_2}{2} \cdot w^2 \cdot \frac{\Delta_z}{2}\right) $
* $a = \mathcal{F}(a)$
* Loop $N=\frac{Z_N}{\Delta_z}$ times:
    * $a = \mathcal{F}^{-1}(a \cdot v_1)$
    * $n\in\mathbb{C}^{N_t} \sim N(0,1)$
    * $a = a + \sqrt{\frac{D\cdot \Delta_z}{dt}}\cdot n$
    * $a = a \cdot \exp\left(j \cdot \gamma \cdot \Delta_z \cdot |a|^2\right)$
    * $a = v_1 \cdot \mathcal{F}(a)$
* $a = \mathcal{F}^{-1}(a)$
* $a = a \cdot \frac{1}{\sqrt{P_0}}$


In [ ]:
ssf = SplitStepFourier(
    b2=-20e-27, # units of SI (s^2/m)
    gamma=0.003,
    t0=125e-12,
    dt=1,
    z_n=1000e3,
    dz=200,
    with_noise=True,
    verbose=True
)

x6 = ssf(x5)

Visualizer.twin_zoom_plot('real X(xi)', np.real(x6), range(4000, 4200))

### 7) NFT
nft back to $ \xi $ domain

In [ ]:
res = nsev(x6, tvec, Xi1=-BW, Xi2=BW, M=N_xi, display_c_msg=True)
assert res['return_value'] == 0, "NFT failed"
x7 = res['cont_ref']  # r[xi,L]

# plot
zm = range(int(N_rrc/2), int(0.5*N_rrc+500))
Visualizer.twin_zoom_plot('real {X(xi)}', np.real(x7), zm, xivec, 'xi[Hz]')
Visualizer.twin_zoom_plot('reference before INFT', np.real(x4),zm, xivec, 'xi[Hz]')


### 8) Equalizer
normalize back
$ \hat X(\xi) = X / A_\mu$

and later on the deep learning model should be placed here


In [ ]:
#descale
    #get amp_phase
amp = np.abs(x7)
phase = np.angle(x7)
    # desigmoid
descaled_amp = -np.log((2 / (amp + 1)) - 1)
    # get complex from amp_phase
x71 = descaled_amp * np.exp(1j * phase)

# normalize
x8 = x71 / normalization_factor

# plot
Visualizer.double_plot('real',np.real(x71),np.real(x8),None,None,'descaled signal','denormalized signal')

### 9) Match Filter
convolve again with the rrc filter and sample every $ N_{over sample}=8 $ points, those points are free from inter symbol interference (aliasing)

In [ ]:
# plot x8, y1 and h_rrc
y1 = np.convolve(x8, h_rrc)
print(f'normalization_factor={normalization_factor}')

Visualizer.double_plot(f'real, mu={normalization_factor}',np.real(x8),np.imag(x8),None,None,'real(x8)','imag{x8}')
Visualizer.double_plot('real',np.real(y1),np.imag(y1),None,None,'real{y1}','imag{y1}')
Visualizer.double_plot('real',np.real(h_rrc),np.imag(h_rrc),None,None,'real{h_rrc}','imag{h_rrc}')


In [ ]:
y1 = np.convolve(x8, h_rrc)

# sampling the analog vector into discrete bits
start = N_rrc
stop = - N_rrc  # + over_sampling
step = over_sampling
x9 = y1[start:stop:step] / over_sampling

Visualizer.twin_zoom_plot('analog signal after another conv (real)',np.real(y1),range(0,2*N_rrc))
Visualizer.twin_zoom_plot('sampled bits (real)',np.real(x9),range(0,50),function='stem')
Visualizer.plot_constellation_map_with_points(x9, m_qam, 'after depulse shaping')
Visualizer.eye_diagram(x9, sps=sps)
print(f'num of sampled symbols = {len(x9)} ')

### 10) demodulation
restore back from i q symbols to bits

In [ ]:
x10 = modem.demodulate(x9)

#plot
Visualizer.print_bits(x10, sps, 'message after channel')

### Evaluate:
calculate ber

In [ ]:
num_errors = (x0 != x10).sum()
ber = num_errors / length_of_msg

print(f'ber = {ber} = {num_errors}/{length_of_msg}')
